In [ ]:
import os
import numpy as np
import skimage.io as io
import skimage.transform as trans
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array, load_img

def dice_loss(y_true, y_pred):
    """
    Dice loss, a measure of overlap between two samples.
    """
    smooth = 1.0  # Smooth factor to avoid division by zero
    y_true_f = keras.flatten(y_true)
    y_pred_f = keras.flatten(y_pred)
    intersection = keras.sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (keras.sum(y_true_f) + keras.sum(y_pred_f) + smooth)

import tensorflow as tf
from tensorflow.keras import backend as K
#metrics
def dice_coefficient(y_true, y_pred):
    smooth = 1e-6  # 为了避免除以零
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def iou(y_true, y_pred):
    smooth = 1e-6
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=[1,2,3])
    iou = (intersection + smooth) / (sum_ - intersection + smooth)
    return iou




def preprocess_image(image_path, target_size=(256, 256)):
    """预处理图片：读取、缩放和归一化"""
    image = load_img(image_path, color_mode='grayscale', target_size=target_size)
    image = img_to_array(image)
    image = np.log1p(image)
    image = (image - np.min(image)) / (np.max(image) - np.min(image))
    image = np.expand_dims(image, axis=0)  # 增加批次维度
    return image

from skimage.transform import resize

def predict_and_save(model, image_path, target_size=(256, 256), original_size=(512, 512), save_path='data/predicted_masks'):
    """对单张图片进行预测，并保存掩码。现在会将预测的掩码调整为原始尺寸。"""
    image = preprocess_image(image_path, target_size)
    mask = model.predict(image)[0]  # 预测并获取掩码
    mask = (mask > 0.5).astype(np.float32)  # 二值化处理，这里改用 float32 以防在 resize 过程中出现问题
    
    # 将掩码调整为原始尺寸
    mask_resized = resize(mask, original_size, mode='constant', preserve_range=True)
    mask_resized = (mask_resized > 0.5).astype(np.uint8) * 255  # 再次二值化处理，确保掩码是黑白的

    file_name = os.path.basename(image_path)
    # 去掉文件名中的 '_band_1' 词缀，并确保保存为PNG格式
    file_name_without_band = file_name.replace('_band_1', '')
    mask_save_path = os.path.join(save_path, os.path.splitext(file_name_without_band)[0] + '_msk.png')
    
    io.imsave(mask_save_path, mask_resized.squeeze())  # 保存掩码图片





#归一化的选择（包括是否使用 log 变换）不直接影响二值化阈值的选择。
#二值化的阈值选择依赖于模型输出的性质和任务的具体要求。
#在实际应用中，0.5 是一个通用的起点，但根据模型性能和特定任务的需求，可能需要调整这个阈值。
#例如，如果您希望减少假阳性的数量，可能会选择一个更高的阈值；反之，如果希望减少假阴性，可能会选择一个更低的阈值。


# 加载模型

# 加载模型时提供自定义损失函数的定义
model = load_model('unet_band1_val_best.hdf5', custom_objects={'dice_loss': dice_loss,'dice_coefficient': dice_coefficient,'iou':iou})

# 指定要预测的图片目录和保存掩码的目录
predict_dir = 'data/filter/image/band_1'
save_dir = 'data/test_band_1'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 遍历目录中的所有图片，对每张图片进行预测并保存掩码
for image_file in os.listdir(predict_dir):
    image_path = os.path.join(predict_dir, image_file)
    predict_and_save(model, image_path, target_size=(256, 256), save_path=save_dir)

print("预测完成，掩码已保存至：", save_dir)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_5336\2152845546.py:64: UserWarning: data/test_band_1\1286_msk.png is a low contrast image
  io.imsave(mask_save_path, mask_resized.squeeze())  # 保存掩码图片
C:\Users\Administrator\AppData\Local\Temp\ipykernel_5336\2152845546.py:64: UserWarning: data/test_band_1\1610_msk.png is a low contrast image
  io.imsave(mask_save_path, mask_resized.squeeze())  # 保存掩码图片
C:\Users\Administrator\AppData\Local\Temp\ipykernel_5336\2152845546.py:64: UserWarning: data/test_band_1\442_msk.png is a low contrast image
  io.imsave(mask_save_path, mask_resized.squeeze())  # 保存掩码图片
C:\Users\Administrator\AppData\Local\Temp\ipykernel_5336\2152845546.py:64: UserWarning: data/test_band_1\640_msk.png is a low contrast image
  io.imsave(mask_save_path, mask_resized.squeeze())  # 保存掩码图片


In [3]:
import os
import zipfile

def zip_images(directory, zip_file_name):
    # 获取指定目录下所有文件的路径
    file_paths = [os.path.join(directory, f) for f in os.listdir(directory)]

    # 初始化 zip 文件
    with zipfile.ZipFile(zip_file_name, 'w') as zipf:
        # 将目录下的所有文件添加到 zip 文件中
        for file_path in file_paths:
            zipf.write(file_path, os.path.basename(file_path))

# 指定目标目录和压缩文件名
target_directory = "data/test_result"
zip_file_name = "data/test_result/dhdata.zip"

# 调用函数进行压缩
zip_images(target_directory, zip_file_name)
